In [ ]:
import pandas as pd
from nn.model_classes import Model, Layer
from nn.functions import mean_square_error, der_mean_square_error, leaky_relu, der_leaky_relu, sigmoid, der_sigmoid, mirror, der_mirror
from nn.trainer import RegressionTrainer
from nn.plotter import Plotter
from nn.dataset_utils import linear_regression_dataset, standardize_data, split_data
from nn.optimizers import SGD, Momentum

In [ ]:
dataset = pd.read_csv(r'datasets\expenses_Sruthi.csv')

In [ ]:
dataset.info()

In [ ]:
dataset.head()

In [ ]:
dataset = pd.get_dummies(dataset, drop_first = True, dtype = int)
dataset.head()

In [ ]:
dataset.info()

In [ ]:
D_train, D_test = split_data(dataset, 0.3, 10)

In [ ]:
D_train.info()

In [ ]:
D_test.info()

In [ ]:
X_train = D_train.drop(columns = ['charges'])
y_train = D_train['charges']

In [ ]:
X_train.head()

In [ ]:
y_train.head()

In [ ]:
model = Model(mean_square_error, der_mean_square_error, 5)
model.add_layer(Layer(X_train.shape[1]))
model.add_layer(Layer(X_train.shape[1], leaky_relu(alpha = 0.4), der_leaky_relu(alpha = 0.4)))
model.add_layer(Layer(X_train.shape[1], leaky_relu(alpha = 0.4), der_leaky_relu(alpha = 0.4)))
model.add_layer(Layer(int(X_train.shape[1] // 1.5), leaky_relu(alpha = 0.4), der_leaky_relu(alpha = 0.4)))
model.add_layer(Layer(1, mirror, der_mirror))
model.compile()

In [ ]:
trainer = RegressionTrainer(model, Momentum(0.5))

In [ ]:
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
y_train = y_train.reshape((y_train.shape[0], 1))
X_means, X_stds = standardize_data(X_train)
y_means, y_stds = standardize_data(y_train)

In [ ]:
trainer.train(X_train, y_train, 1, 0.02, 50)

In [ ]:
trainer.save_history('./logs', 'dataset_regression')

In [ ]:
plotter = Plotter()
plotter.read_file(r'logs\dataset_regression.txt')
plotter.plot_gradients("./plots", "dataset_regression", 700)
plotter.plot_weights("./plots", "dataset_regression", 700)
plotter.plot_score("./plots", "dataset_regression", 700, False)

In [ ]:
X_test = D_test.drop(columns = ['charges'])
y_test = D_test['charges']

In [ ]:
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()
y_test = y_test.reshape((y_test.shape[0], 1))

In [ ]:
standardize_data(X_test, X_means, X_stds)
standardize_data(y_test, y_means, y_stds)

In [ ]:
_ = trainer.predict(X_test, y_test)